<a href="https://colab.research.google.com/github/thandziuk1991/PyTorch-Tutorials/blob/main/mnist_cnn_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils
from torch.utils.data import DataLoader,SubsetRandomSampler
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import numpy as np

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
training_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testing_data = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)


In [ ]:
plt.imshow(testing_data[1978][0].reshape(28,28))

In [68]:
## let's split training data 80% / 20% to get train/validation split

indices = list(range(len(training_data)))
np.random.seed(25)
np.random.shuffle(indices)
ratio = 0.8
spl_index = int(np.floor(ratio*len(training_data)))

train_index,valid_index = indices[:spl_index],indices[spl_index:]
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)

train_data = DataLoader(training_data,batch_size = 15,sampler = train_sampler)
validation_data = DataLoader(training_data,batch_size = 15,sampler=valid_sampler)


In [69]:
### build a cnn
class CNNModel(nn.Module):
  def __init__(self):
    super().__init__()
    ## convolutional layers
    self.conv1 = nn.Conv2d(1,5,3,1) ### 1 input, 5 features, 3x3 filter, stride = 1
    self.conv2 = nn.Conv2d(5,5,3,1)
    ## fully connected layers
    self.fc1 = nn.Linear(5*5*3,100)
    self.fc2 = nn.Linear(100,50)
    self.fc3 = nn.Linear(50,10) ### last layer, output are 10 digits

  def forward(self,X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,1)
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,1)

    ### flatten out
    X = X.view(-1,5*5*3)

    ### pass to fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X,dim = 1)





In [65]:
### define loss/optimizers
torch.manual_seed(25)
model = CNNModel()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [71]:
epochs = 5

train_loss = []
validation_loss = []
train_correct = []
validation_correct = []


for i in range(epochs):

  trn_correct = 0
  validation_crt = 0

  for b,(X_train,y_train) in enumerate(train_data):

    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)

    predicted = torch.max(y_pred.data,1)[1]
    batch_correct = (predicted == y_train).sum()

    trn_correct += batch_correct

    ### update params

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss.append(loss)
  train_correct.append(trn_correct)

  ### validation

  with torch.no_grad():

    for b,(X_valid,y_valid) in enumerate(validation_data):
      y_pred_val = model(X_valid)
      loss = criterion(y_pred_val,y_valid)

      predicted = torch.max(y_pred_val.data,1)[1]
      batch_correct = (predicted == y_pred_val).sum()

      validation_crt += batch_correct

  validation_loss.append(loss)
  validation_correct.append(validation_crt)









ValueError: Expected input batch_size (484) to match target batch_size (15).